In [7]:
#reference: following code are modified from the tutorial note
def build_linear_regression_model():
    #import data
    datasets = ['fifa.npz','finance.npz','orbits.npz']
    for dataset in datasets:
        with np.load(dataset) as data:
            a = data['train_X']
            b = data['regression_train_Y']
            c = data['test_X']
            d = data['regression_test_Y']
        df_TrainX = pd.DataFrame(a)
        df_TrainRegY = pd.DataFrame(b)
        df_TestX = pd.DataFrame(c)
        df_TestRegY = pd.DataFrame(d)
        #train the model
        start_time = time.time()
        LinearRegressionModel = LinearRegression()
        LinearRegressionModel.fit(df_TrainX,df_TrainRegY)
        LinearRegressionModel_predTestY = LinearRegressionModel.predict(df_TestX)
        LinearRegressionModel_predTrainY = LinearRegressionModel.predict(df_TrainX)
        #computing R2 score 
        R2_test = r2_score(df_TestRegY.to_numpy(),LinearRegressionModel_predTestY)
        R2_train = r2_score(df_TrainRegY.to_numpy(),LinearRegressionModel_predTrainY)
        print("R2 score for %s test set: " % (dataset),R2_test)
        print("R2 score for %s train set: " % (dataset),R2_train)
        end_time = time.time()
        print("Overall running time (wall time) for %s:" % (dataset), round(end_time - start_time, 3))
        #calculate squared error and produce the histogram
        error = (LinearRegressionModel_predTestY - df_TestRegY)**2
        print("sum of squared loss: ",error.sum(axis=0))
        plt.hist(error.to_numpy(),bins='auto')
        plt.title('Squared error for test set %s' %(dataset))
        plt.xlabel('Squared error')
        plt.ylabel('Frequency')
        plt.show()
        print()

In [41]:
#reference: following code are modified from the tutorial note
def build_logistic_regression_model():
    #import data 
    datasets = ['fifa.npz','finance.npz','orbits.npz']
    for dataset in datasets:
        #fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,10))
        with np.load(dataset) as data:
            a = data['train_X']
            b = data['test_X']
            c = data['classification_train_Y']
            d = data['classification_test_Y']
        df_TrainX = pd.DataFrame(a)
        df_TestX = pd.DataFrame(b)
        df_TrainClassY = pd.DataFrame(c)
        df_TestClassY = pd.DataFrame(d)
        #train the model
        start_time = time.time()
        LogisticRegressionModel = SGDClassifier(loss = 'log',max_iter=100,tol=1e-4,random_state=4211)
        LogisticRegressionModel.fit(df_TrainX,df_TrainClassY.values.ravel())
        LogisticRegressionModel_predTestY = LogisticRegressionModel.predict(df_TestX)
        LogisticRegressionModel_predTrainY = LogisticRegressionModel.predict(df_TrainX)
        end_time = time.time()
        test_accuracy = LogisticRegressionModel.score(df_TestX,df_TestClassY)
        train_accuracy = LogisticRegressionModel.score(df_TrainX,df_TrainClassY)
        print("Test accuracy for %s test set: " % (dataset), test_accuracy)
        print("Test accuracy for %s train set: " % (dataset), train_accuracy)
        print("Overall running time (wall time) for %s:" % (dataset), round(end_time - start_time, 3))
        #drawing the confusion matrix
        cm1 = confusion_matrix(df_TestClassY.to_numpy(), LogisticRegressionModel_predTestY)
        cm2 = confusion_matrix(df_TrainClassY.to_numpy(), LogisticRegressionModel_predTrainY)
        plot_cm(cm1,cm2,dataset)
        

In [42]:
def plot_cm(cm1,cm2,title):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,10))
    im1 = ax1.imshow(cm1, interpolation='nearest', cmap='Set3')
    ax1.set_title('Confusion matrix for %s test set' %(title), size = 10)
    ax1.figure.colorbar(im1,ax=ax1)
    tick_marks = np.arange(2)
    ax1.set_xticks(tick_marks)
    ax1.set_xticklabels(["0", "1"])
    ax1.set_yticks(tick_marks)
    ax1.set_yticklabels(["0", "1"])
    ax1.set_ylabel('Actual label', size = 10)
    ax1.set_xlabel('Predicted label', size = 10)
    width, height = cm1.shape
    for x in range(width):
        for y in range(height):
            ax1.annotate(str(cm1[x][y]), xy=(y, x),
                         horizontalalignment='center',
                         verticalalignment='center')
    im2 = ax2.imshow(cm2, interpolation='nearest', cmap='Set3')
    ax2.set_title('Confusion matrix for %s train set' %(title), size = 10)
    ax2.figure.colorbar(im2,ax=ax2)
    ax2.set_xticks(tick_marks)
    ax2.set_xticklabels(["0", "1"])
    ax2.set_yticks(tick_marks)
    ax2.set_yticklabels(["0", "1"])
    ax2.set_ylabel('Actual label', size = 10)
    ax2.set_xlabel('Predicted label', size = 10)
    width, height = cm2.shape
    for x in range(width):
        for y in range(height):
            ax2.annotate(str(cm2[x][y]), xy=(y, x), 
            horizontalalignment='center',
            verticalalignment='center')
    plt.show()

In [47]:
def find_best_H(trainX,trainY):
    rand = [10,20,30,40,50]
    units = [1,2,3,4,5,6,7,8,9,10]
    result = []
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,10))
    start_time = time.time()
    for unit in units:
        mlp = MLPClassifier(max_iter=1000, alpha=1e-3, solver='sgd', verbose=False, tol=1e-4, random_state=4211,
                             learning_rate='adaptive',learning_rate_init=0.01,hidden_layer_sizes = (unit,))
        temp_result_validation = []
        temp_result_train = []
        for i in range(5):
            X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size = 0.2, random_state = rand[i])
            mlp.fit(X_train, Y_train.values.ravel())
            #check for accuracy
            test_score = accuracy_score(Y_test,mlp.predict(X_test))
            train_score = accuracy_score(Y_train,mlp.predict(X_train))
            print('test score for round %d: ' %(i+1),test_score)
            print('train score for round %d: ' %(i+1),train_score)
            temp_result_validation.append(test_score)
            temp_result_train.append(train_score)
        print('loss : ',mlp.loss_)
        avg_result = sum(temp_result_validation)/5
        print("The score for %d hidden unit: " %(unit), avg_result)
        print()
        result.append(avg_result)
        #plot the performance change
        ax1.set_title("validation set")
        ax1.plot([1,2,3,4,5],temp_result_validation,label='H=%d' %(unit))
        ax1.legend(loc='best')
        ax1.set_xlabel('Train number')
        ax1.set_ylabel('accuracy')
        ax2.set_title("train set")
        ax2.plot([1,2,3,4,5],temp_result_train,label='H=%d' %(unit))
        ax2.legend(loc='best')
        ax2.set_xlabel('Train number')
        ax2.set_ylabel('accuracy')
    #find the best H
    index, best_accuracy = max(enumerate(result), key=operator.itemgetter(1))
    end_time = time.time()
    print("Overall running time (wall time) for cross validation:", round(end_time - start_time, 3))
    index += 1
    print('best H: ',index)
    print('best_accuracy: ',best_accuracy)
    print()
    plt.subplots_adjust(wspace=0.3)
    plt.show()
    return index

In [50]:
#reference: following code are modified from the tutorial note
def build_neural_network():
    #import data 
    datasets = ['fifa.npz','finance.npz','orbits.npz']
    #for dataset in datasets:
    for dataset in datasets:
        with np.load(dataset) as data:
            a = data['train_X']
            b = data['test_X']
            c = data['classification_train_Y']
            d = data['classification_test_Y']
        df_TrainX = pd.DataFrame(a)
        df_TestX = pd.DataFrame(b)
        df_TrainClassY = pd.DataFrame(c)
        df_TestClassY = pd.DataFrame(d)
        print("Find best H for dataset %s" %(dataset))
        best_H = find_best_H(df_TrainX,df_TrainClassY)
        #retrain the neural network with the best H
        start_time = time.time()
        print("best H found: ", best_H)
        best_mlp = MLPClassifier(hidden_layer_sizes= (best_H,),max_iter=1000, alpha=1e-3, solver='sgd', 
                                 verbose=False, tol=1e-4,random_state=4211,learning_rate='adaptive',learning_rate_init=0.01)
        best_mlp.fit(df_TrainX,df_TrainClassY.values.ravel())
        end_time = time.time()
        print("Overall running time (wall time) to retrain the neural network:", round(end_time - start_time, 3))
        test_accuracy = best_mlp.score(df_TestX,df_TestClassY)
        train_accuracy = best_mlp.score(df_TrainX,df_TrainClassY)
        print("Test accuracy for %s test set: " % (dataset), test_accuracy)
        print("Test accuracy for %s train set: " % (dataset), train_accuracy)
        print("loss: ",best_mlp.loss_)
        predTestY = best_mlp.predict(df_TestX)
        predTrainY = best_mlp.predict(df_TrainX)
        cm1 = confusion_matrix(df_TestClassY.to_numpy(), predTestY)
        cm2 = confusion_matrix(df_TrainClassY.to_numpy(), predTrainY)
        plot_cm(cm1,cm2,dataset)

In [ ]:
#main function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import operator
from sklearn.linear_model import LinearRegression,SGDClassifier
from sklearn.model_selection import train_test_split  #,GridSearchCV,cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,accuracy_score,log_loss,r2_score,mean_squared_error,confusion_matrix



#build_linear_regression_model()
#build_logistic_regression_model()
build_neural_network()